In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set the style for better looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelsize'] = 12

# 1.1 Loading data
df = pd.read_csv("train.csv")
print("Dataset shape:", df.shape)
print("Column names:", df.columns.tolist())
df.head()

# 1.2 Check for missing values and process them
print("Missing values in each column:")
print(df.isnull().sum())

# Handle missing values
df['Возраст'] = df['Возраст'].fillna(df['Возраст'].median())
df = df.drop('Номер каюты', axis=1)
df['Порт отправления'] = df['Порт отправления'].fillna(df['Порт отправления'].mode()[0])

print("\nAfter handling missing values:")
print(df.isnull().sum())

# 2.1 Histogram of passenger age distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['Возраст'], bins=30, kde=True)
plt.title('Распределение возраста пассажиров Титаника')
plt.xlabel('Возраст (лет)')
plt.ylabel('Количество пассажиров')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 2.2 Boxplot comparing passenger ages by cabin class
plt.figure(figsize=(10, 6))
sns.boxplot(x='класс', y='Возраст', data=df)
plt.title('Сравнение возраста пассажиров по классам кают')
plt.xlabel('Класс каюты')
plt.ylabel('Возраст (лет)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 2.3 Pie chart to visualize the proportion of survivors and fatalities
plt.figure(figsize=(8, 8))
survival_counts = df['Выжил'].value_counts()
labels = ['Не выжили', 'Выжили']
colors = ['#ff6b6b', '#4ecdc4']
explode = (0.1, 0)  # explode the 1st slice

plt.pie(survival_counts, labels=labels, colors=colors, autopct='%1.1f%%',
        startangle=90, shadow=True, explode=explode)
plt.title('Соотношение выживших и погибших на Титанике')
plt.axis('equal')
plt.tight_layout()
plt.show()

# 2.4 Linear graph of the dependence of the number of passengers on age
plt.figure(figsize=(12, 6))

# Create age bins and count passengers in each bin
age_bins = np.arange(0, df['Возраст'].max() + 5, 5)
age_groups = pd.cut(df['Возраст'], bins=age_bins)
passenger_counts = age_groups.value_counts().sort_index()

# Create line plot
sns.lineplot(x=age_bins[:-1], y=passenger_counts.values, marker='o',
             linewidth=2.5, markersize=8)
plt.title('Зависимость количества пассажиров от возраста')
plt.xlabel('Возраст (лет)')
plt.ylabel('Количество пассажиров')
plt.xticks(age_bins)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3.1 Pairplot for multiple numeric columns
numeric_columns = ['Возраст', 'Стоимость проезда пассажира',
                   '# братьев и сестер / супругов на борту',
                   '# родителей/детей на борту']

print("Создание парного графика...")
pair_grid = sns.pairplot(df[numeric_columns], diag_kind='kde', corner=True,
                         plot_kws={'alpha': 0.6})
pair_grid.fig.suptitle('Парный график числовых переменных Титаника',
                       fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# 3.2 Scatterplot with trendline to analyze the relationship between Age and Fare
plt.figure(figsize=(10, 6))
sns.regplot(x='Возраст', y='Стоимость проезда пассажира', data=df,
            scatter_kws={'alpha':0.5}, line_kws={'color':'red', 'linewidth':2})
plt.title('Зависимость стоимости билета от возраста пассажира')
plt.xlabel('Возраст (лет)')
plt.ylabel('Стоимость проезда ($)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Additional visualization: Survival rate by class and gender
plt.figure(figsize=(12, 6))
sns.barplot(x='класс', y='Выжил', hue='Гендер', data=df, errorbar=None)
plt.title('Уровень выживаемости по классу каюты и полу')
plt.xlabel('Класс каюты')
plt.ylabel('Доля выживших')
plt.legend(title='Пол', loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
numeric_df = df[['Выжил', 'класс', 'Возраст', '# братьев и сестер / супругов на борту',
                 '# родителей/детей на борту', 'Стоимость проезда пассажира']]
correlation_matrix = numeric_df.corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, fmt='.2f', mask=mask,
            cbar_kws={'label': 'Коэффициент корреляции'})
plt.title('Тепловая карта корреляций числовых переменных Титаника')
plt.tight_layout()
plt.show()

# Additional: Family size analysis
df['Размер семьи'] = df['# братьев и сестер / супругов на борту'] + df['# родителей/детей на борту']

plt.figure(figsize=(10, 6))
sns.countplot(x='Размер семьи', data=df)
plt.title('Распределение пассажиров по размеру семьи')
plt.xlabel('Размер семьи')
plt.ylabel('Количество пассажиров')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Survival rate by family size
plt.figure(figsize=(12, 6))
family_survival = df.groupby('Размер семьи')['Выжил'].mean().reset_index()
sns.barplot(x='Размер семьи', y='Выжил', data=family_survival, errorbar=None)
plt.title('Уровень выживаемости в зависимости от размера семьи')
plt.xlabel('Размер семьи')
plt.ylabel('Доля выживших')
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()